<a href="https://colab.research.google.com/github/JesperHLund/ML_mini_project/blob/main/CIFAR10_MLP_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MLP classifier trained on the Fashion MNIST dataset and tuned using KerasTuner**

# Image classification using the Fashion MNIST dataset
We will train and evaluate an MLP on the Fashion MNIST dataset. It consists of 70.000 grayscale images of 28x28 pixels each, and there are 10 classes.
We will use KerasTuner for hyperparameter Tuning.

## Setup

In [ ]:
# Common imports
import sys
import os
import tensorflow as tf
from tensorflow import keras

# to make this notebook's output stable across runs
tf.random.set_seed(42)

# To plot figures
%matplotlib inline
import matplotlib.pyplot as plt

## Load the data

In [ ]:
data = tf.keras.datasets.cifar10
(X_train_full, y_train_full), (X_test, y_test) = data.load_data()


In [ ]:
# Show the size and dimension of the dataset.
X_train_full.shape

(50000, 32, 32, 3)

In [ ]:
# Each pixel intensity is represented as a byte (0 to 255).
X_train_full.dtype

dtype('uint8')

In [ ]:
# Split the full training set into a validation set and a training set,
# and scale the pixel intensities down to the 0-1 range and convert them to floats, by dividing by 255.
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

## Hyperparameter tuning with KerasTuner

In [ ]:
# Installing KerasTuner
if "google.colab" in sys.modules:
    %pip install -q -U keras_tuner

In [ ]:
# Create a function that builds, compiles and returns a Keras model.
import keras_tuner as kt

def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model

Using TensorFlow backend


In [ ]:
# EarlyStopping with rollback to the best model.
early_stopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

# Performance scheduling, multiply the learning rate by a factor when the error stops dropping
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=4)

# Use KerasTuner RandomSearch to tune hyperparameters
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=10, overwrite=True,
    directory="my_mnist", project_name="my_rnd_search", seed=42)
random_search_tuner.search(X_train, y_train, epochs=100,
                           validation_data=(X_valid, y_valid), callbacks=[lr_scheduler, early_stopping])

Trial 10 Complete [00h 15m 27s]
val_accuracy: 0.5564000010490417

Best val_accuracy So Far: 0.5605999827384949
Total elapsed time: 01h 44m 31s


In [ ]:
# Display a summary of the best model:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 03 summary
Hyperparameters:
n_hidden: 2
n_neurons: 251
learning_rate: 0.001715074355925934
Score: 0.5605999827384949


In [ ]:
# Display the best model's accuracy measured on the validation set:
best_trial.metrics.get_last_value("val_accuracy")

0.5605999827384949

In [ ]:
# Evaluate the model's accuracy on the test set:
best_model = random_search_tuner.get_best_models(num_models=1)[0]
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)

313/313 [==============================] - 2s 6ms/step - loss: 1.3175 - accuracy: 0.5507
